In [16]:
import pandas as pd

csv = pd.read_csv("inventors_disj_new.csv")


In [17]:
import copy

new_csv = copy.deepcopy(csv)

for i, rec in csv.iterrows():
    stmt = rec['statement']
    city_start = len("It is the case either that ")
    city_end = stmt.index("lived")
    # print(stmt)
    # print(stmt[city_start:city_end])

    or_that = stmt.index("or that ") + 8
    after_it = stmt[or_that:]
    # stmt = stmt[:or_that] + stmt[city_start:city_end] + after_it
    stmt = stmt[:city_start] + "the inventor " + stmt[city_start:or_that] + "the inventor " + after_it
    
    new_csv.loc[i, ['statement']] = copy.copy(stmt)


In [18]:
new_csv['statement'][3]

'It is the case either that the inventor Tim Berners-Lee lived in Germany or that the inventor Tim Berners-Lee lived in the U.S.'

In [19]:
new_csv.to_csv("inventors_disj_new.csv", index=False)

In [5]:
import pandas as pd

neg_companies = pd.read_csv("neg_companies.csv")
neg_companies = pd.DataFrame({
    'statement': neg_companies['statement'].tolist(),
    'label': [1-l for l in neg_companies['label'].tolist()],
})


In [7]:
neg_companies.to_csv("neg_companies.csv")

In [16]:
import pandas as pd

gpt_results = pd.read_csv("factuality_judgment.csv")
companies = pd.read_csv("companies.csv")


In [17]:
import numpy as np

gpt_results = np.array([int(i) for i in gpt_results['label'].tolist()])
companies = np.array([int(i) for i in companies['label'].tolist()])


In [21]:
p_o = np.count_nonzero(gpt_results == companies) / len(gpt_results)
p_yes = np.count_nonzero(gpt_results==1)/len(gpt_results) * np.count_nonzero(companies==1)/len(companies)
p_no = np.count_nonzero(gpt_results==0)/len(gpt_results) * np.count_nonzero(companies==0)/len(companies)
p_e = p_yes + p_no
kappa = (p_o-p_e) / (1-p_e)
kappa


0.020000000000000018

In [22]:
np.count_nonzero(companies==gpt_results)/len(companies)

0.51

In [ ]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import torch
from transformers import (
    AutoModelForCausalLM, AutoTokenizer,
    QuantoConfig
)



DEVICE = "cuda"

MODEL_PATH = "/Model/meta-llama/Meta-Llama-3.1-8B-Instruct-hf"

model_name = os.path.basename(MODEL_PATH)

tok = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=False)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
tok.padding_side = 'left'

quant_config = QuantoConfig(weights='float8')
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    device_map='auto',
    attn_implementation="eager",
    quantization_config=quant_config
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
csv = pd.read_csv("element_symb_disj.csv")

answers = []
for i, rec in tqdm(csv.iterrows(), total=len(csv)):
    stmt = rec['statement']
    msg = [
        {"role": "system", "content": """You are a helpful assistant. When asked to judge the factuality of a statement, you need to consider the statement as a whole.
For example, when asked \"Is the following statement true or false?\nIt is the case either that Molybdenum has the symbol Mo or that Molybdenum has the symbol In.\", you should answer \"True\", because Molybdenum has the symbol Mo, even if it does not has the symbol In."""},
        {"role": "user", "content": "Is the following statement true or false?\n" + stmt},
    ]
    ids = tok.apply_chat_template(msg, tokenize=False, add_generation_prompt=True)
    ids = tok(ids, return_tensors='pt').to(DEVICE)
    outputs = model.generate(**ids, do_sample=False, max_new_tokens=32, pad_token_id=tok.eos_token_id, top_p=None, temperature=None, top_k=None)
    ans = tok.decode(outputs[0, ids.input_ids.shape[1]:], skip_special_tokens=True).lower().strip(" \n")
    answers.append(0 if "false" in ans else 1)
    # print(ans, rec['label'])
    # if i == 2:
    #     break


In [40]:
print(tok.decode(outputs[0]))

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Is the following statement true or false? Answer a single word "true" or "false".

either 'Niobium has the symbol Nb' or 'Niobium has the symbol Cl' is true.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

False<|eot_id|>


In [ ]:
outfile = pd.DataFrame({
    "statement": csv['statement'].tolist(),
    "label": answers
})

In [9]:
outfile.to_csv("element_symb_disj_llama3_8.csv", index=False)

In [6]:
def kappa(x, y):
    p_o = np.count_nonzero(x == y) / len(x)
    p_yes = np.count_nonzero(x==1)/len(x) * np.count_nonzero(y==1)/len(y)
    p_no = np.count_nonzero(x==0)/len(x) * np.count_nonzero(y==0)/len(y)
    p_e = p_yes + p_no
    return (p_o-p_e) / (1-p_e)


In [64]:
original_answers = np.array(csv['label'].tolist())
llama_answers = np.array(answers)

np.count_nonzero(original_answers==llama_answers)/original_answers.size, kappa(original_answers, llama_answers)

(0.9767441860465116, 0.9518477043673012)

## SciQ

In [ ]:
from datasets import load_dataset
import numpy as np

dataset_path = "/Dataset/allenai/sciq"
dataset = load_dataset(dataset_path)
dataset['test'][0]


{'question': 'Compounds that are capable of accepting electrons, such as o 2 or f2, are called what?',
 'distractor3': 'residues',
 'distractor1': 'antioxidants',
 'distractor2': 'Oxygen',
 'correct_answer': 'oxidants',
 'support': 'Oxidants and Reductants Compounds that are capable of accepting electrons, such as O 2 or F2, are calledoxidants (or oxidizing agents) because they can oxidize other compounds. In the process of accepting electrons, an oxidant is reduced. Compounds that are capable of donating electrons, such as sodium metal or cyclohexane (C6H12), are calledreductants (or reducing agents) because they can cause the reduction of another compound. In the process of donating electrons, a reductant is oxidized. These relationships are summarized in Equation 3.30: Equation 3.30 Saylor URL: http://www. saylor. org/books.'}

In [4]:
dataset['validation'][0]

{'question': 'Who proposed the theory of evolution by natural selection?',
 'distractor3': 'Scopes',
 'distractor1': 'Linnaeus',
 'distractor2': 'shaw',
 'correct_answer': 'darwin',
 'support': ''}

In [3]:
template = """Context: {context}
Question: {question}
Options:
- {c1}
- {c2}
- {c3}
- {c4}
Answer: {answer}"""
sciq_true_false = []
rng = np.random.RandomState(0)
few_shot_rng = np.random.RandomState(1)
n_shots = 3

def get_few_shot_prompt(few_shot_dataset, rng, n_shots):
    few_shot_indices = rng.choice(len(few_shot_dataset), n_shots, replace=False).tolist()
    few_shot_prompt = ""
    for i, ind in enumerate(few_shot_indices):
        rec = few_shot_dataset[ind]
        choices = ['correct_answer', 'distractor1', 'distractor2', 'distractor3']
        rng.shuffle(choices)
        if i != -1:
            text = template.format(question=rec['question'], context=rec['support'], answer=rec['correct_answer'], c1=rec[choices[0]], c2=rec[choices[1]], c3=rec[choices[2]], c4=rec[choices[3]])
        else:
            text = template.format(question=rec['question'], context=rec['support'], answer=rec['distractor1'], c1=rec[choices[0]], c2=rec[choices[1]], c3=rec[choices[2]], c4=rec[choices[3]])
        few_shot_prompt += text + "\n\n"
    return few_shot_prompt


for rec in dataset['test']:
    choices = ['correct_answer', 'distractor1', 'distractor2', 'distractor3']
    rng.shuffle(choices)
    
    sciq_true_false.append({
        "statement": get_few_shot_prompt(dataset['validation'], few_shot_rng, n_shots) + template.format(question=rec['question'], context=rec['support'], answer=rec['correct_answer'], c1=rec[choices[0]], c2=rec[choices[1]], c3=rec[choices[2]], c4=rec[choices[3]]),
        "label": 1,
    })
    for i in range(1, 4):
        rng.shuffle(choices)
        sciq_true_false.append({
            "statement": get_few_shot_prompt(dataset['validation'], few_shot_rng, n_shots) + template.format(question=rec['question'], context=rec['support'], answer=rec[f'distractor{i}'], c1=rec[choices[0]], c2=rec[choices[1]], c3=rec[choices[2]], c4=rec[choices[3]]),
            "label": 0,
        })


In [7]:
template = """Context: {context}
Question: {question}
Options:
A. {c1}
B. {c2}
C. {c3}
D. {c4}
Answer: {answer}"""
sciq_true_false = []
rng = np.random.RandomState(0)
few_shot_rng = np.random.RandomState(1)
n_shots = 3

def get_few_shot_prompt(few_shot_dataset, rng, n_shots):
    few_shot_indices = rng.choice(len(few_shot_dataset), n_shots, replace=False).tolist()
    few_shot_prompt = ""
    for i, ind in enumerate(few_shot_indices):
        rec = few_shot_dataset[ind]
        choices = ['correct_answer', 'distractor1', 'distractor2', 'distractor3']
        rng.shuffle(choices)
        if i >= 2:
            text = template.format(question=rec['question'], context=rec['support'], answer=chr(65+choices.index('correct_answer')), c1=rec[choices[0]], c2=rec[choices[1]], c3=rec[choices[2]], c4=rec[choices[3]])
        else:
            text = template.format(question=rec['question'], context=rec['support'], answer=chr(65 + choices.index('distractor1')), c1=rec[choices[0]], c2=rec[choices[1]], c3=rec[choices[2]], c4=rec[choices[3]])
        few_shot_prompt += text + "\n\n"
    return few_shot_prompt


for rec in dataset['test']:
    choices = ['correct_answer', 'distractor1', 'distractor2', 'distractor3']
    rng.shuffle(choices)
    
    fs_prompt = get_few_shot_prompt(dataset['validation'], few_shot_rng, n_shots)
    # fs_prompt = ""
    sciq_true_false.append({
        "statement": fs_prompt + template.format(question=rec['question'], context=rec['support'], answer=chr(65+choices.index('correct_answer')), c1=rec[choices[0]], c2=rec[choices[1]], c3=rec[choices[2]], c4=rec[choices[3]]),
        "label": 1,
    })
    for i in range(1, 4):
        rng.shuffle(choices)
        fs_prompt = get_few_shot_prompt(dataset['validation'], few_shot_rng, n_shots)
        # fs_prompt = ""
        sciq_true_false.append({
            "statement": fs_prompt + template.format(question=rec['question'], context=rec['support'], answer=chr(65 + choices.index(f'distractor{i}')), c1=rec[choices[0]], c2=rec[choices[1]], c3=rec[choices[2]], c4=rec[choices[3]]),
            "label": 0,
        })


In [8]:
print(sciq_true_false[0]['statement'])

Context: Recall that boiling occurs when the vapor pressure of a liquid is equal to the atmospheric pressure. Since adding a solute lowers the vapor pressure, we would expect a higher temperature to be required before boiling can begin. This phenomenon, known as boiling point elevation , occurs whenever a solute is dissolved into a pure solvent.
Question: What occurs when the vapor pressure of a liquid is equal to the atmospheric pressure?
Options:
A. freezing
B. boiling
C. melting
D. evaporation
Answer: A

Context: The pea plant Pisum sativum has purple and white flowers. These flowered plants are not just pretty to look at. These plants led Gregor Mendel to unlock the secrets of heredity, beginning the field of genetics. For his efforts, Mendel is widely known as the Father of Genetics, even though he knew nothing of the genetic material, DNA. The laws he developed apply to all sexually reproducing life, and are the basis for beginning to understand many human diseases.
Question: Who

In [9]:
import os
import pandas as pd

sciq_csv = pd.DataFrame({
    "statement": [rec['statement'] for rec in sciq_true_false],
    "label": [rec['label'] for rec in sciq_true_false],
})
sciq_csv.to_csv("sciq_true_false_mc_FFT.csv", index=False)


## MMLU

In [ ]:
import os
import pandas as pd
import numpy as np

mmlu_base_path = "/Dataset/mmlu"

def load_mmlu_split(mmlu_base_path, split):
    mmlu_split_path = os.path.join(mmlu_base_path, split)
    ds = {}
    for f in os.listdir(mmlu_split_path):
        task = f.replace(".csv", "").replace(f"_{split}", "")
        csv = pd.read_csv(os.path.join(mmlu_split_path, f), names=("question", "A", "B", "C", "D", "answer"))
        ds[task] = csv
    return ds

dev_set = load_mmlu_split(mmlu_base_path, "dev")
test_set = load_mmlu_split(mmlu_base_path, "test")


In [61]:
TEMPLATE_NO_OPTIONS = """Question: {question}
Answer: {answer}"""

TEMPLATE_WITH_OPTIONS = """Question: {question}
Options:
- {c1}
- {c2}
- {c3}
- {c4}
Answer: {answer}"""

mmlu_true_false = []
n_shots = 5
select_rng = np.random.RandomState(42)
few_shot_rng = np.random.RandomState(0)

def get_few_shot_prompt(few_shot_dataset, rng, n_shots):
    few_shot_indices = rng.choice(len(few_shot_dataset), n_shots, replace=False).tolist()
    few_shot_prompt = ""
    for i, ind in enumerate(few_shot_indices):
        rec = few_shot_dataset.iloc[ind]
        if i <= 1:
            text = TEMPLATE_WITH_OPTIONS.format(question=rec['question'], answer=rec[rec['answer']], c1=rec['A'], c2=rec['B'], c3=rec['C'], c4=rec['D'])
        else:
            text = TEMPLATE_WITH_OPTIONS.format(question=rec['question'], answer=rec[chr(ord('A') + ((ord(rec['answer'])-ord('A')+1)%4))], c1=rec['A'], c2=rec['B'], c3=rec['C'], c4=rec['D'])
        few_shot_prompt += text + "\n\n"
    return few_shot_prompt


for task, csv in test_set.items():
    csv = csv.iloc[select_rng.choice(len(csv), 50, replace=False)]
    for _, rec in csv.iterrows():
        for i in range(4):
            choice = chr(ord('A')+i)
            text = TEMPLATE_WITH_OPTIONS.format(question=rec['question'], c1=rec['A'], c2=rec['B'], c3=rec['C'], c4=rec['D'], answer=rec[choice])
            few_shot_prompt = get_few_shot_prompt(dev_set[task], few_shot_rng, n_shots)
            text = few_shot_prompt + text
            mmlu_true_false.append({
                "statement": text,
                "label": 1 if choice == rec['answer'] else 0,
            })


In [7]:
TEMPLATE_MC = """Question: {question}
A. {c1}
B. {c2}
C. {c3}
D. {c4}
Answer: {answer}"""

mmlu_true_false = []
n_shots = 5
select_rng = np.random.RandomState(42)
few_shot_rng = np.random.RandomState(0)

def get_few_shot_prompt(few_shot_dataset, rng, n_shots):
    few_shot_indices = rng.choice(len(few_shot_dataset), n_shots, replace=False).tolist()
    few_shot_prompt = ""
    for i, ind in enumerate(few_shot_indices):
        rec = few_shot_dataset.iloc[ind]
        if i <= 5:
            text = TEMPLATE_MC.format(question=rec['question'], answer=rec['answer'], c1=rec['A'], c2=rec['B'], c3=rec['C'], c4=rec['D'])
        else:
            text = TEMPLATE_MC.format(question=rec['question'], answer=chr(ord('A') + ((ord(rec['answer'])-ord('A')+1)%4)), c1=rec['A'], c2=rec['B'], c3=rec['C'], c4=rec['D'])
        few_shot_prompt += text + "\n\n"
    return few_shot_prompt


for task, csv in test_set.items():
    csv = csv.iloc[select_rng.choice(len(csv), 50, replace=False)]
    for _, rec in csv.iterrows():
        for i in range(4):
            choice = chr(ord('A')+i)
            text = TEMPLATE_MC.format(question=rec['question'], c1=rec['A'], c2=rec['B'], c3=rec['C'], c4=rec['D'], answer=choice)
            few_shot_prompt = get_few_shot_prompt(dev_set[task], few_shot_rng, n_shots)
            text = few_shot_prompt + text
            mmlu_true_false.append({
                "statement": text,
                "label": 1 if choice == rec['answer'] else 0,
            })


In [8]:
print(mmlu_true_false[0]['statement'])

Question: From the solubility rules, which of the following is true?
A. All chlorides, bromides, and iodides are soluble
B. All sulfates are soluble
C. All hydroxides are soluble
D. All ammonium-containing compounds are soluble
Answer: D

Question: Which of the following is considered an acid anhydride?
A. HCl
B. H2SO3
C. SO2
D. Al(NO3)3
Answer: C

Question: Which of the following is expected to be a polar molecule?
A. PCl4F
B. BF3
C. CO2
D. Si(CH3)4
Answer: A

Question: A new compound is synthesized and found to be a monoprotic acid with a molar mass of 248 g/mol. When 0.0050 mol of this acid are dissolved in 0.500 L of water, the pH is measured as 3.89. What is the pKa of this acid?
A. 3.89
B. 7.78
C. 5.78
D. 2.33
Answer: C

Question: A solution contains 2.00 mole of acetic acid, CH3COOH, and 1.00 mole of calcium acetate, Ca(CH3COO)2. The solution is able to resist the addition of a small amount of strong acid or strong base with only minor changes in the pH of the solution. Larger q

In [9]:
len(mmlu_true_false)

11400

In [10]:
csv = pd.DataFrame({
    "statement": [rec['statement'] for rec in mmlu_true_false],
    "label": [rec['label'] for rec in mmlu_true_false],
})
csv.to_csv("mmlu_true_false_mc_TTTTT.csv", index=False)


## TriviaQA

In [2]:
import json
import os

model_name = "Meta-Llama-3.1-8B-hf"
# model_name = "Llama-2-13b-hf"
results_file = f"triviaqa/results/{model_name}-seed=0-temperature=1.0-top_k=50-num_seq=20-shots=20.jsonl"

answers = []
with open(results_file) as fp:
    for line in fp:
        answers.append(json.loads(line))


In [ ]:
from datasets import load_dataset

dataset = load_dataset("/Dataset/mandarjoshi/trivia_qa")
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'],
        num_rows: 138384
    })
    validation: Dataset({
        features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'],
        num_rows: 17944
    })
    test: Dataset({
        features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'],
        num_rows: 17210
    })
})

In [4]:
import numpy as np
import re, string

FEW_SHOT_TEMPLATE = """Question: {question}
Answer: {answer}"""

GENERATION_TEMPLATE = """Question: {question}
Answer:"""

triviaqa_true_false = []
n_shots = 5
few_shot_rng = np.random.RandomState(0)

def get_few_shot_prompt(few_shot_dataset, template, rng, n_shots):
    few_shot_indices = rng.choice(len(few_shot_dataset), n_shots, replace=False).tolist()
    few_shot_prompt = ""
    for i, ind in enumerate(few_shot_indices):
        rec = few_shot_dataset[ind]
        text = template.format(question=rec['question'], answer=rec['answer']['normalized_value'])
        few_shot_prompt += text + "\n\n"
    return few_shot_prompt


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def handle_punc(text):
        exclude = set(string.punctuation + "".join([u"‘", u"’", u"´", u"`"]))
        return ''.join(ch if ch not in exclude else ' ' for ch in text)

    def lower(text):
        return text.lower()

    def replace_underscore(text):
        return text.replace('_', ' ')

    return white_space_fix(remove_articles(handle_punc(lower(replace_underscore(s))))).strip()


def has_exact_match(ground_truths, candidates):
    for ground_truth in ground_truths:
        if ground_truth in candidates:
            return True
    return False


for i in range(len(answers)):
    few_shot_prompt = get_few_shot_prompt(dataset['train'], FEW_SHOT_TEMPLATE, few_shot_rng, n_shots)
    rec = dataset['validation'][i]
    for ans in answers[i]:
        prompt = few_shot_prompt + FEW_SHOT_TEMPLATE.format(question=rec['question'], answer=ans)
        normalized_ans = normalize_answer(ans)
        label = 1 if any(a in normalized_ans for a in rec['answer']['normalized_aliases']) else 0
        triviaqa_true_false.append({
            "statement": prompt,
            "label": label,
        })


In [5]:
import pandas as pd

csv = pd.DataFrame({
    "statement": [rec['statement'] for rec in triviaqa_true_false],
    "label": [rec['label'] for rec in triviaqa_true_false],
})
csv.to_csv(f"triviaqa_true_false_{model_name}-shots=20.csv", index=False)


## Xsum

In [ ]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

from datasets import load_dataset

xsum = load_dataset("/Dataset/xsum")
xsum

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [18]:
xsum['test'][0]

{'document': 'Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.\nWorkers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.\nThe Welsh Government said more people than ever were getting help to address housing problems.\nChanges to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.\nPrison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.\nHowever, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.\nAndrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, said the

In [ ]:
from transformers import AutoTokenizer

MODEL_PATH = "/Model/meta-llama/Meta-Llama-3.1-8B-hf"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=False)
MAX_LENGTH = 8192


In [ ]:
import pandas as pd

hallu_csv = pd.read_csv("/Dataset/xsum_hallucination/hallucination_annotations_xsum_summaries.csv")
ids = set(hallu_csv['bbcid'])
hallu_selected = []
for _, rec in hallu_csv.iterrows():
    if rec['bbcid'] in ids:
        for orig in xsum['test']:
            if orig['id'] == str(rec['bbcid']):
                break
        hallu_selected.append({
            'id': rec['bbcid'],
            'document': orig['document'],
            'correct_summary': orig['summary'],
            'wrong_summary': rec['summary'],
        })
        ids.remove(rec['bbcid'])



In [21]:
hallu_selected[0]

{'id': 34687720,
 'document': 'France\'s Dubuisson carded a 67 to tie with overnight leader Van Zyl of South Africa on 16 under par.\nMcIlroy carded a third straight five under-par 67 to move to 15 under par with Thailand\'s Kiradech Aphibarnrat.\nThe world number three\'s round included an eagle on the 12th as he bids to win his first title since May.\n"The 67s I\'ve shot this week have all been a little different and I feel like I\'ve played within myself for all of them, " said four-time major winner McIlroy of Northern Ireland. "I feel there\'s a low round out there for me and hopefully it\'s tomorrow."\nMcIlroy was level par for the day after 10 holes, dropping his first shots of the week by three-putting the third and 10th, the latter mistake prompting the 26-year-old to throw his putter at his bag.\nBut he hit back with a birdie on the par-five 11th and a towering four iron from 229 yards on the 13th set up an eagle from just four feet.\nThe former world number one ruptured a li

In [22]:
import numpy as np

XSUM_TEMPLATE = """Summarize this document: {doc}
Summary: {sum}"""

xsum_true_false = []
n_shots = 2
few_shot_rng = np.random.RandomState(0)

def get_few_shot_prompt(few_shot_dataset, rng, n_shots):
    few_shot_indices = rng.choice(len(few_shot_dataset), n_shots, replace=False).tolist()
    few_shot_prompt = ""
    for i, ind in enumerate(few_shot_indices):
        rec = few_shot_dataset[ind]
        text = XSUM_TEMPLATE.format(doc=rec['document'].replace("\n", " "), sum=rec['summary'])
        few_shot_prompt += text + "\n\n"
    return few_shot_prompt


for rec in hallu_selected:
    # pos_stmt = XSUM_TEMPLATE.format(doc=rec['document'].replace("\n", " "), sum=rec['correct_summary'])
    # neg_stmt = XSUM_TEMPLATE.format(doc=rec['document'].replace("\n", " "), sum=rec['wrong_summary'])
    
    few_shot_prompt = get_few_shot_prompt(xsum['train'], few_shot_rng, n_shots)
    pos_stmt = few_shot_prompt + XSUM_TEMPLATE.format(doc=rec['document'].replace("\n", " "), sum=rec['correct_summary'])
    neg_stmt = few_shot_prompt + XSUM_TEMPLATE.format(doc=rec['document'].replace("\n", " "), sum=rec['wrong_summary'])
    if len(tokenizer.encode(pos_stmt)) > MAX_LENGTH or len(tokenizer.encode(neg_stmt)) > MAX_LENGTH:
        continue
    
    xsum_true_false.append({
        "statement": pos_stmt,
        "label": 1,
    })
    xsum_true_false.append({
        "statement": neg_stmt,
        "label": 0,
    })

print(len(xsum_true_false))


Token indices sequence length is longer than the specified maximum sequence length for this model (14724 > 8192). Running this sequence through the model will result in indexing errors


998


In [23]:
print(xsum_true_false[0]['statement'])

Summarize this document: It said that dissidents targeted an army vehicle with an improvised explosive device. Four other soldiers were wounded. The attack has been condemned by President Juan Manuel Santos, who last year won the Nobel Peace Prize for his work on the peace agreement. The deal ended half a century of conflict but renegade Farc fighters have since clashed with other Farc members as well as government forces. Colombia landslide: Farc rebels offer to help rebuild town The Farc rebel rapping for peace Dozens of rights activists killed in Colombia in 2016 The government and the Farc signed a peace agreement in November. Most Farc members are now inside or heading to one of 26 transition camps dotted around the country. They were set up as part of the historic peace deal which ended more than five decades of armed uprising. But some Farc leaders have complained that, despite months of planning, most of the transition camps are still lacking the basic amenities they say they w

In [24]:
import pandas as pd

csv = pd.DataFrame({
    "statement": [rec['statement'] for rec in xsum_true_false],
    "label": [rec['label'] for rec in xsum_true_false],
})
csv.to_csv("xsum_true_false_TT.csv", index=False)


## BoolQ

In [ ]:
from datasets import load_dataset

boolq = load_dataset("/Dataset/google/boolq")
boolq


DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 3270
    })
})

In [2]:
boolq['train'][0]

{'question': 'do iran and afghanistan speak the same language',
 'answer': True,
 'passage': 'Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.'}

In [12]:
import numpy as np

BOOLQ_TEMPLATE = """Passage: {passage}
Question: {question}?
Options:
- Yes
- No
Answer: {answer}"""


boolq_true_false = []
n_shots = 1
few_shot_rng = np.random.RandomState(0)

def get_few_shot_prompt(few_shot_dataset, rng, n_shots):
    few_shot_indices = rng.choice(len(few_shot_dataset), n_shots, replace=False).tolist()
    few_shot_prompt = ""
    for i, ind in enumerate(few_shot_indices):
        rec = few_shot_dataset[ind]
        if i <= -100:
            text = BOOLQ_TEMPLATE.format(passage=rec['passage'], question=rec['question'], answer="Yes" if rec['answer'] else "No")
        else:
            text = BOOLQ_TEMPLATE.format(passage=rec['passage'], question=rec['question'], answer="No" if rec['answer'] else "Yes")
        few_shot_prompt += text + "\n\n"
    return few_shot_prompt


for rec in boolq['validation']:
    # prompt = BOOLQ_TEMPLATE.format(passage=rec['passage'], question=rec['question'], answer="Yes" if rec['answer'] else "No")
    few_shot_prompt = get_few_shot_prompt(boolq['train'], few_shot_rng, n_shots)
    prompt = few_shot_prompt + BOOLQ_TEMPLATE.format(passage=rec['passage'], question=rec['question'], answer="Yes" if rec['answer'] else "No")
    
    boolq_true_false.append({
        "statement": prompt,
        "label": 1,
    })
    
    # prompt = BOOLQ_TEMPLATE.format(passage=rec['passage'], question=rec['question'], answer="No" if rec['answer'] else "Yes")
    prompt = few_shot_prompt + BOOLQ_TEMPLATE.format(passage=rec['passage'], question=rec['question'], answer="No" if rec['answer'] else "Yes")
    boolq_true_false.append({
        "statement": prompt,
        "label": 0,
    })


In [15]:
print(boolq_true_false[0]['statement'])

Passage: Firearms regulations are uniform throughout Florida, and a carry license is valid everywhere other than in a few specially-defined areas. These prohibited places include any police station, prison, courthouse, polling place, government meeting place, airport, seaport, or tavern. Concealed carry is also prohibited in any school, except for authorized security personnel or armed marshals.
Question: can you carry a concealed weapon in florida?
Options:
- Yes
- No
Answer: No

Passage: All biomass goes through at least some of these steps: it needs to be grown, collected, dried, fermented, distilled, and burned. All of these steps require resources and an infrastructure. The total amount of energy input into the process compared to the energy released by burning the resulting ethanol fuel is known as the energy balance (or ``energy returned on energy invested''). Figures compiled in a 2007 report by National Geographic Magazine point to modest results for corn ethanol produced in t

In [16]:
import pandas as pd

csv = pd.DataFrame({
    "statement": [rec['statement'] for rec in boolq_true_false],
    "label": [rec['label'] for rec in boolq_true_false],
})
csv.to_csv("boolq_true_false_with_options_F.csv", index=False)
